In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorboard.plugins.hparams import api as hp

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test =x_test.reshape(x_test.shape[0], 28, 28, 1)

In [ ]:

HP_ACTIVATION_1 = hp.HParam('activation1', hp.Discrete(['relu', 'elu','selu']))
HP_ACTIVATION_2 = hp.HParam('activation2', hp.Discrete(['relu', 'elu','selu']))
HP_ACTIVATION_3 = hp.HParam('activation3', hp.Discrete(['relu', 'elu','selu']))
HP_ACTIVATION_4 = hp.HParam('activation4', hp.Discrete(['relu', 'elu','selu']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_ACTIVATION_1,HP_ACTIVATION_2,HP_ACTIVATION_3,HP_ACTIVATION_4,HP_ACTIVATION_5,HP_ACTIVATION_6],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
def train_test_model(hparams):
  model = keras.Sequential([
    keras.layers.Conv2D( 32, kernel_size = (5,5),padding = 'Same', input_shape = (28,28,1)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_1]),
    keras.layers.Conv2D( 32, kernel_size = (5,5),padding = 'Same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_2]),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.30),
    keras.layers.Conv2D(64, [3, 3],padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_3]),
    keras.layers.Conv2D(64, [3, 3],padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation(hparams[HP_ACTIVATION_4]),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.30),
    
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.30),
    keras.layers.Dense(10, activation='softmax'),
    
])
  model.compile(
      optimizer='NAdam',
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=10) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [ ]:
accuracy1=[]
combination =0
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    global combination
    combination+=1
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    accuracy1.append(['combination_'+str(combination),list(hparams.values())[0],list(hparams.values())[1],list(hparams.values())[2],list(hparams.values())[3],accuracy])
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    

In [ ]:
hparams.values()

dict_values(['relu', 'elu', 'relu', 'elu'])

In [ ]:
session_num = 0

# for num_units in HP_NUM_UNITS.domain.values:
#   for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
for activation1 in HP_ACTIVATION_1.domain.values:
  for activation2 in HP_ACTIVATION_2.domain.values:
    for activation3 in HP_ACTIVATION_3.domain.values:
      for activation4 in HP_ACTIVATION_4.domain.values:
        hparams = {
                  
                  HP_ACTIVATION_1: activation1,
                  HP_ACTIVATION_2: activation2,
                  HP_ACTIVATION_3: activation3,
                  HP_ACTIVATION_4: activation4,
              }
        run_name = activation1
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
              
        session_num += 1


--- Starting trial: elu
{'activation1': 'elu', 'activation2': 'elu', 'activation3': 'elu', 'activation4': 'elu'}
Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.4790 - accuracy: 0.8310
Epoch 2/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3248 - accuracy: 0.8835
Epoch 3/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2862 - accuracy: 0.8961
Epoch 4/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2563 - accuracy: 0.9084
Epoch 5/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2354 - accuracy: 0.9141
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2138 - accuracy: 0.9211
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2015 - accuracy: 0.9258
Epoch 8/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1860 - accuracy: 0.9317
Epoch 9/10
1875/1875 [==============================] - 13s 7ms

In [ ]:
df = pd.DataFrame(accuracy1,columns=('combination no','Activation1','Activation2','Activation3','Activation4', 'Accuracy'))
df

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)
column = df["Accuracy"]
max_value = column.idxmax()



,combination no,Activation1,Activation2,Activation3,Activation4,Accuracy
0,combination_1,elu,elu,elu,elu,0.9276
1,combination_2,elu,elu,elu,relu,0.9269
2,combination_3,elu,elu,elu,selu,0.9264
3,combination_4,elu,elu,relu,elu,0.9294
4,combination_5,elu,elu,relu,relu,0.9264
5,combination_6,elu,elu,relu,selu,0.9263
6,combination_7,elu,elu,selu,elu,0.9264
7,combination_8,elu,elu,selu,relu,0.9148
8,combination_9,elu,elu,selu,selu,0.9299
9,combination_10,elu,relu,elu,elu,0.9274


In [ ]:
print("The best model combination is as follows:\n")
df.iloc[max_value,:]

The best model combination is as follows:



combination no    combination_68
Activation1                 selu
Activation2                 relu
Activation3                 relu
Activation4                 relu
Accuracy                  0.9337
Name: 67, dtype: object

In [ ]:
df.to_pickle("./abhishek2b.pkl")

In [ ]:
from google.colab import files
files.download('./abhishek2b.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df1= pd.read_pickle('/content/2b.pkl')
df1.iloc[max_value,:]

combination no    combination_68
Activation1                 selu
Activation2                 relu
Activation3                 relu
Activation4                 relu
Accuracy                  0.9337
Name: 67, dtype: object

In [ ]:
%tensorboard --logdir logs/hparam_tuning